In [ ]:
from cassandra.cluster import Cluster
import station_pb2_grpc
import station_pb2
import pandas as pd
import sys
import os
import subprocess
import grpc
from datetime import date

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import corr
from pyspark.sql.functions import col
import matplotlib.pyplot as plt

In [ ]:
# Download the datasets

In [ ]:
try:
    cluster = Cluster(['p6_db_2', 'p6_db_1', 'p6_db_3'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [ ]:
spark = (SparkSession.builder
         .appName("txt-reader")
         .config("spark.driver.allowMultipleContexts", "true")
         .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.2.0")
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

## Part 1

In [ ]:
def setup_cassandra_table():
    # TODO: Q1
    raise NotImplementedError()

In [ ]:
# Q1 Ans
setup_cassandra_table()
print(session.execute("describe keyspace weather").one().create_statement)
print(session.execute("describe type weather.station_record").one().create_statement)
print(session.execute("describe table weather.stations").one().create_statement)